In [1]:
import pickle
import pykitti
import pydriver
import numpy as np

Reading the drive cloud.

In [2]:
def read_clouds(basedir='/media/sf_thesis', date='2011_09_26', drive='0020'):
    data = pykitti.raw(basedir, date, drive, frames=range(0, 2))
    for cloud in data.velo:
        yield cloud[:,:3]

In [3]:
cloud_gen = read_clouds()

cloud_1 = pydriver.pcl.pcl.PCLHelper()
cloud_2 = pydriver.pcl.pcl.PCLHelper()

cl1_data, cl2_data = next(cloud_gen), next(cloud_gen)
cloud_1.fromArray(cl1_data)
cloud_2.fromArray(cl2_data)

In [6]:
index_from = pickle.load(open('index_from.pkl', 'rb'))
index_to = pickle.load(open('index_to.pkl', 'rb'))
distances = pickle.load(open('distances.pkl', 'rb'))

In [ ]:
# Adding colors
max_color = max(distances)
colors = np.zeros((len(index_from), 3), dtype=np.uint8)
colors[:,2] = np.array(np.array(distances) / max_color * 255, dtype=np.uint8)


In [17]:
# Processing later
keypoints_1, keypoints_2 = pydriver.pcl.pcl.PCLHelper(RGB=colors), pydriver.pcl.pcl.PCLHelper()
keypoints_1.fromArray(cl1_data[index_from])
keypoints_2.fromArray(cl2_data[index_to])

In [18]:
# Visualizing

In [ ]:
normal_vectors

In [22]:
# normal vectors

normal_vectors = cl2_data[index_to] - cl1_data[index_from]
normal = pydriver.pcl.pcl.PCLHelper(XYZ=normal_vectors)

In [ ]:
cloud_1.visualizeKeypoints(keypoints_1, normals=normal)

In [6]:
def show(cloud):
    cloud.visualize()

In [7]:
def extractSHOT(cloud, shotRadius=3.0):
    extractor = pydriver.features.shot.SHOTExtractor(shotRadius=shotRadius)
    return extractor.getFeatures({'cloud': cloud}, cloud)

In [8]:
feats_1, feats_2 = extractSHOT(cloud_1), extractSHOT(cloud_2)

In [9]:
from tqdm import tqdm

def match_keypoints(array_1, array_2, feats_1, feats_2):
    kp_1, kp_2 = [], []
    for _ in tqdm(range(min(len(feats_1), len(feats_2)))):
        index = np.random.choice(np.arange(len(feats_1)))
        corr_index = np.argmin(np.sum((feats_2 - feats_1[index]) ** 2, axis=-1))
        kp_1.append(array_1[index])
        kp_2.append(array_2[corr_index])
        
        def delete(arr, index):
            arr = np.delete(arr, index, axis=0)

        delete(array_1, index)
        delete(feats_1, index)
        delete(array_2, corr_index)
        delete(feats_2, corr_index)
    return kp_1, kp_2

In [10]:
import cv2 as cv

In [11]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params,search_params)


In [12]:
cf1, cf2 = list(map(list, feats_1[1])), list(map(list, feats_2[1]))

In [13]:
np.array(cf2)

array([[0.00187842, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.00172382,
        0.41934082],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00173811,
        0.42104608],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00196267,
        0.42047116]], dtype=float32)

In [14]:
matches = flann.knnMatch(np.array(cf1), np.array(cf2), k=1)

In [15]:
distances = [match[0].distance for match in matches]

In [16]:
query = [match[0].queryIdx for match in matches]
to = [match[0].trainIdx for match in matches]

In [17]:
FEATURE_DIST_THRESHOLD = 0.5
for i in reversed(range(len(distances))):
    if distances[i] > FEATURE_DIST_THRESHOLD:
        del distances[i]
        del query[i]
        del to[i]

In [18]:
len(distances)

108333

In [19]:
max(query), max(to)

(122481, 122141)

In [20]:
index_from, index_to = map(np.array, [query, to])

In [25]:
import pickle
pickle.dump(index_from, open('index_from.pkl', 'wb'))
pickle.dump(index_to, open('index_to.pkl', 'wb'))
pickle.dump(distances, open('distances.pkl', 'wb'))

In [19]:
keypoints_1, keypoints_2 = pydriver.pcl.pcl.PCLHelper(), pydriver.pcl.pcl.PCLHelper()
keypoints_1.fromArray(cloud_1.toArray(False, False)[index_from])
keypoints_2.fromArray(cloud_2.toArray(False, False)[index_to])

MemoryError: 

In [70]:
kp1, kp2 = match_keypoints(cloud_1.toArray(False, False).copy()[0], cloud_2.toArray(False, False).copy()[0],
                          feats_1[1].copy(), feats_2[1].copy())


  0%|          | 20/122142 [00:08<15:24:35,  2.20it/s]

KeyboardInterrupt: 

In [49]:
feats_2[0]

array([(56.911,  8.037,  2.153,  2.5517917),
       (57.196,  8.26 ,  2.163,  2.4592474),
       (56.772,  8.29 ,  2.149,  2.5931783), ...,
       ( 3.67 , -1.388, -1.714, -3.058455 ),
       ( 3.676, -1.377, -1.715, -3.0574098),
       ( 3.711, -1.377, -1.73 , -3.052723 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('rotation_y', '<f4')])

# Keypoint matching

In [17]:
import pydriver


In [22]:
extractor = pydriver.features.shot.SHOTExtractor(shotRadius=3.0)

In [24]:
result = extractor.getFeatures({'cloud': helper}, helper)

In [27]:
helper.toArray(False, False)

array([[[57.389,  7.92 ,  2.168],
        [57.472,  8.024,  2.171],
        [57.589,  8.225,  2.176],
        ...,
        [ 3.675, -1.39 , -1.716],
        [ 3.709, -1.389, -1.731],
        [ 3.606, -1.337, -1.677]]], dtype=float32)

In [32]:
result[1][1]

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 9.8310673e-05,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e

In [9]:
extractor = pydriver.keypoints.base.KeypointExtractor()

In [10]:
preprocessor = pydriver.preprocessing.preprocessing.Preprocessor()

In [12]:
featureExtractor = pydriver.features.SHOTColorExtractor(shotRadius=2.0)
# keypoints = helper.copy()

In [12]:
helper.visualizeKeypoints(keypoints)

In [10]:
keypoints = ppcl.PCLHelper()

In [11]:
keypoints.fromArray(cloud_1[30000:-30000,:3])

In [11]:
SCENE = preprocessor.process(FRAME)

NameError: name 'FRAME' is not defined

In [12]:
extractor.getKeypointCloud(SCENE)

NameError: name 'SCENE' is not defined